# UNIÓN DE TABLAS EDLIST - ECHA - PESTICIDAS - COSING
El objetivo de este archivo es la unión de tablas y la creación de una tabla única con el nombre que aparece en las etiquetas

Las tablas que se van a unir son:

- **edlist_clean** -> contiene disrupores endocrinos y nombres químicos
- **echa_clean** -> contiene disrupores endocrinos y nombres químicos
- **pesticidas** -> contiene el listados de las sustancias activas de pesticidas 
- **cosing_clean** -> contiene ingredientes y sustancias usadas en productos cosméticos y nombres de las etiquetas 

La primera unión -> **edlist_clean** y **echa_clean** se harán por las claves: CAS no. y EC no. 

La segunda unión -> **edlist_echa** y **pesticidas** se harán por las claves: CAS no. y EC no.

La tercera unión -> **edlist_echa_pesticidas** y **cosing_clean** se harán por las claves: CAS no. y EC no. 

**IMPORTANTE:** En adelante cuando nos refiramos al **nombre INCI** nos referimos al nombre de la etiqueta de los productos, porque es como se le conoce de manera internacional

## 1. Importamos librerias y definimos funciones

In [1]:
import pandas as pd
import numpy as np

**DEFINIMOS FUNCIONES**

In [2]:
def nombre_mas_informativo_generico(df, col1, col2, nueva_col):
    def elegir_mas_informativo(row):
        val1 = row[col1]
        val2 = row[col2]

        if pd.isna(val1):
            return val2
        if pd.isna(val2):
            return val1
        if val1 == val2:
            return val1
        return val1 if len(str(val1)) > len(str(val2)) else val2

    df[nueva_col] = df.apply(elegir_mas_informativo, axis=1)
    return df

## 2. Importamos Edlist_clean

In [3]:
# leemos los datos de la lista de EDlist_clean
edlist_clean= pd.read_parquet("../../data/processed/notebooks/edlist_clean.parquet")

In [4]:
edlist_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          239 non-null    int64 
 1   Name and abbreviation  239 non-null    object
 2   Health effects         239 non-null    int64 
 3   Environmental effects  239 non-null    int64 
 4   Status                 239 non-null    object
 5   Regulatory Field       225 non-null    object
 6   Year                   239 non-null    int64 
 7   Appears on lists       239 non-null    object
 8   Fuente_original        239 non-null    object
 9   CAS Number             230 non-null    object
 10  EC Number              195 non-null    object
dtypes: int64(4), object(7)
memory usage: 20.7+ KB


In [5]:
# Consulatamos si hay al menos un None en string en alguna columna
edlist_clean.isin([None]).any()

ID (for name)            False
Name and abbreviation    False
Health effects           False
Environmental effects    False
Status                   False
Regulatory Field          True
Year                     False
Appears on lists         False
Fuente_original          False
CAS Number                True
EC Number                 True
dtype: bool

In [6]:
# Reemplazamos None por NaN
edlist_clean = edlist_clean.replace({None: np.nan})

In [7]:
edlist_clean['Name and abbreviation'].nunique(dropna=True)

209

Hay 209 disruptores en esta lista

## 3. Importamos ECHA_clean

In [8]:
# leemos los datos de la lista de ECHA_clean
echa_clean= pd.read_parquet("../../data/processed/notebooks/echa_clean.parquet")

In [9]:
echa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Substance name   150 non-null    object
 1   EC Number        122 non-null    object
 2   CAS Number       145 non-null    object
 3   Authority        149 non-null    object
 4   Status           150 non-null    object
 5   Outcome          150 non-null    object
 6   Latest update    150 non-null    object
 7   First published  150 non-null    object
 8   Fuente_original  150 non-null    object
dtypes: object(9)
memory usage: 10.7+ KB


In [10]:
# Consulatamos si hay al menos un None en string en alguna columna
echa_clean.isin(["None"]).any()

Substance name     False
EC Number          False
CAS Number         False
Authority          False
Status             False
Outcome            False
Latest update      False
First published    False
Fuente_original    False
dtype: bool

In [11]:
echa_clean['Substance name'].nunique(dropna=True)

149

Hay 149 disruptores en esta lista

## 4. Primera unión de dataframes

La unión que se llevará a cabo es un full outer join, es decir queremos todo de ambas tablas, porque así ganamos registros 

In [12]:
edlist_echa = edlist_clean.merge( echa_clean, on=['CAS Number', 'EC Number'],  how='outer', suffixes=('', '_ECHA'))

In [13]:
edlist_echa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID (for name)          239 non-null    float64
 1   Name and abbreviation  239 non-null    object 
 2   Health effects         239 non-null    float64
 3   Environmental effects  239 non-null    float64
 4   Status                 239 non-null    object 
 5   Regulatory Field       225 non-null    object 
 6   Year                   239 non-null    float64
 7   Appears on lists       239 non-null    object 
 8   Fuente_original        239 non-null    object 
 9   CAS Number             282 non-null    object 
 10  EC Number              250 non-null    object 
 11  Substance name         150 non-null    object 
 12  Authority              149 non-null    object 
 13  Status_ECHA            150 non-null    object 
 14  Outcome                150 non-null    object 
 15  Latest

Como hay dos columnas que informan de la fuente original pero de las distintas tablas vamos a combinar "Fuente_original" y "Fuente_original_ECHA" 

In [14]:
# Combinamos para que si "Fuente_original" está vacío, se rellene con "Fuente_original_ECHA"
edlist_echa['Fuente_original'] = edlist_echa['Fuente_original'].combine_first(edlist_echa['Fuente_original_ECHA'])

# Eliminar la columna sobrante
edlist_echa.drop(columns=['Fuente_original_ECHA'], inplace=True)


In [15]:
print(edlist_echa['Fuente_original'].info())
edlist_echa['Fuente_original'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 295 entries, 0 to 294
Series name: Fuente_original
Non-Null Count  Dtype 
--------------  ----- 
295 non-null    object
dtypes: object(1)
memory usage: 2.4+ KB
None


Fuente_original
Edlist_1                130
Edlist_2                 93
ECHA - ED_pendiente      46
Edlist_3                 16
ECHA - ED_Humans&Env      6
ECHA - ED_Env             3
ECHA - ED_Humans          1
Name: count, dtype: int64

De los 295 registros hay disrupotores que aun no se han confirmado y están en evaluación:

* Edlist_2                 93
* ECHA - ED_pendiente      46

## 5. Comprobaciones del Merge

Consultamos los datos para ver si la unión se ha hecho correctamente.

Para ello comprobaresmos los campos "Name and abbreviation" proveniente de la EDlist y el "Substance name" proveniente de la ECHA, si ambos están rellenos los nombres tienen que coincidir o ser muy similares en los siguientes filtros:

1. En los registros que cumplen la condición de que 'CAS no.' y 'EC no.' ambos sean NO NULOS, 
2. En los registros que cumplen la condición de que 'CAS no.' sea NO NULO y que 'EC no.' sean  NULOS
3. En los registros que cumplen la condición de que 'CAS no.' sea NULO y que 'EC no.' sean NO NULOS
4. En los registros que cumplen la condición de que 'CAS no.' y 'EC no.' ambos sean NULOS --> no tiene que haber registro con esta condición porque hemos eliminado esta condición de ambas listas en sus respectivos ETLs

In [16]:
#Consulta del número de registros que cumple la condición 1
Num_consulta_1= edlist_echa[(edlist_echa['CAS Number'].notna()) & (edlist_echa['EC Number'].notna())]
Num_consulta_1[['CAS Number', 'EC Number']].count()

CAS Number    237
EC Number     237
dtype: int64

In [17]:
# Consulta del listado de algunos de los registros de la condición 1
edlist_echa[(edlist_echa['CAS Number'].notna()) & (edlist_echa['EC Number'].notna())].tail(3)

,ID (for name),Name and abbreviation,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original,CAS Number,EC Number,Substance name,Authority,Status_ECHA,Outcome,Latest update,First published
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ECHA - ED_pendiente,98-95-3,202-716-0,Nitrobenzene,Austria,Concluded,postponed,05-nov-2021,12-oct-2018
280,376.0,"Butanoic acid, 4-amino-4-oxosulfo-, N-coco alk...",1.0,1.0,Corap List,REACH,2023.0,List II,Edlist_2,98171-53-0,308-662-5,"Butanoic acid, 4-amino-4-oxosulfo-, N-coco alk...",France,Under development under SEV,Under development (SEV),19-jul-2023,19-jul-2023
281,177.0,Methyl 4-hydroxybenzoate (Methylparaben),1.0,1.0,Corap List,REACH,2014.0,List II,Edlist_2,99-76-3,202-785-7,Methyl 4-hydroxybenzoate,France,Under development under SEV,Under development (SEV),03-feb-2021,12-oct-2018


In [18]:
#Consulta del número de registros que cumple la condición 2
Num_consulta_2= edlist_echa[(edlist_echa['CAS Number'].notna()) & (edlist_echa['EC Number'].isna())]
Num_consulta_2[['CAS Number', 'EC Number']].count()

CAS Number    45
EC Number      0
dtype: int64

In [19]:
# Consulta de algunos de los registros de la condición 2
edlist_echa[(edlist_echa['CAS Number'].notna()) & (edlist_echa['EC Number'].isna())].tail(2)

,ID (for name),Name and abbreviation,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original,CAS Number,EC Number,Substance name,Authority,Status_ECHA,Outcome,Latest update,First published
266,305.0,"4-(4,4-dimethylpentan-2-yl)phenol",0.0,1.0,Environmental candidate list,REACH,2017.0,List I,Edlist_1,911371-07-8,NaN,"4-(4,4-dimethylpentan-2-yl)phenol",Austria,Concluded,ED ENV,03-feb-2021,04-oct-2018
274,360.0,"(1R,3E,4S)-1,7,7-trimethyl-3-(4- methylbenzyli...",1.0,0.0,Health candidate list,REACH,2022.0,List I,Edlist_1,95342-41-9,NaN,"(1R,3E,4S)-1,7,7-trimethyl-3-(4-methylbenzylid...",Denmark,Concluded,ED HH,04-ago-2022,20-oct-2020


In [20]:
#Consulta del número de registros que cumple la condición 3
Num_consulta_3= edlist_echa[(edlist_echa['CAS Number'].isna()) & (edlist_echa['EC Number'].notna())]
Num_consulta_3[['CAS Number', 'EC Number']].count()

CAS Number     0
EC Number     13
dtype: int64

In [21]:
# Consulta de algunos de los registros de la condición 3
edlist_echa[(edlist_echa['CAS Number'].isna()) & (edlist_echa['EC Number'].notna())].tail(3)

,ID (for name),Name and abbreviation,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original,CAS Number,EC Number,Substance name,Authority,Status_ECHA,Outcome,Latest update,First published
292,334.0,"Nonylphenol, ethoxylated (EO = 4)",0.0,1.0,Environmental authorisation list,REACH,2017.0,List I,Edlist_1,NaN,939-975-0,NaN,NaN,NaN,NaN,NaN,NaN
293,335.0,"Nonylphenol, ethoxylated (EO = 10)",0.0,1.0,Environmental authorisation list,REACH,2017.0,List I,Edlist_1,NaN,939-993-9,NaN,NaN,NaN,NaN,NaN,NaN
294,246.0,"Esterification products of 1,3-dioxo-2-benzofu...",1.0,1.0,Corap List,REACH,2027.0,List II,Edlist_2,NaN,941-303-6,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Consulta de algunos de los registros de la condición 4
edlist_echa[(edlist_echa['CAS Number'].isna()) & (edlist_echa['EC Number'].isna())]

,ID (for name),Name and abbreviation,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original,CAS Number,EC Number,Substance name,Authority,Status_ECHA,Outcome,Latest update,First published


Este dataframe tiene 303 registros de disruptores endrocinos. Y las consultas que hemos hecho nos dan el total de esos 303 (245+45+13)

In [23]:
# Consulta tipo de nombres distintos
diferentes_names = edlist_echa[
    (edlist_echa['Name and abbreviation'].notna()) &
    (edlist_echa['Substance name'].notna()) &
    (edlist_echa['Name and abbreviation'] != edlist_echa['Substance name'])
]
diferentes_names[['CAS Number', 'EC Number', 'Name and abbreviation', 'Substance name']].count()

CAS Number               34
EC Number                29
Name and abbreviation    34
Substance name           34
dtype: int64

In [24]:
diferentes_names[['Name and abbreviation', 'Substance name' ]].head(10)

,Name and abbreviation,Substance name
24,"2,2-dibromo-2-cyanoacetamide (DBNPA)","2,2-dibromo-2-cyanoacetamide"
32,Resorcinol,"resorcinol; 1,3-benzenediol"
42,Triphenyl phosphate (TPP),Triphenyl phosphate
46,"2,2′,6,6′-Tetra-tert-butyl-4,4′- methylenediph...","2,2',6,6'-tetra-tert-butyl-4,4'-methylenediphenol"
51,"1,3,4,6,7,8- hexahydro-4,6,6,7,8,8- hexamethyl...","1,3,4,6,7,8-hexahydro-4,6,6,7,8,8-hexamethylin..."
53,Reaction products of phosphoryl trichloride an...,Reaction products of phosphoryl trichloride an...
57,"2,6-di-tert-butyl-p-cresol; butylated hydroxyt...","2,6-di-tert-butyl-p-cresol"
63,Oxybenzone (BP-3),Oxybenzone
79,"1,7,7-trimethyl-3-[(4-methylphenyl)methylene]b...","1,7,7-trimethyl-3-(phenylmethylene)bicyclo[2.2..."
141,Diuron,"diuron (ISO); 3-(3,4-dichlorophenyl)-1,1-dimet..."


In [25]:
# Seleccionamos el nombre más informativo
nombre_mas_informativo_generico(edlist_echa, 'Name and abbreviation', 'Substance name', 'Name_Edlist_Echa')

edlist_echa [['Name and abbreviation', 'Substance name', 'Name_Edlist_Echa']].iloc[20:30]

,Name and abbreviation,Substance name,Name_Edlist_Echa
20,Cyclomethicone,NaN,Cyclomethicone
21,Cyclomethicone,NaN,Cyclomethicone
22,4-(5-methylhexyl)phenol,4-(5-methylhexyl)phenol,4-(5-methylhexyl)phenol
23,Triclocarban,NaN,Triclocarban
24,"2,2-dibromo-2-cyanoacetamide (DBNPA)","2,2-dibromo-2-cyanoacetamide","2,2-dibromo-2-cyanoacetamide (DBNPA)"
25,4-(3-methylhexyl)phenol,4-(3-methylhexyl)phenol,4-(3-methylhexyl)phenol
26,2-(4-nonylphenoxy)ethanol,NaN,2-(4-nonylphenoxy)ethanol
27,p-nonylphenol,NaN,p-nonylphenol
28,3-aminopropyldiethylamine,3-aminopropyldiethylamine,3-aminopropyldiethylamine
29,NaN,Chloramide,Chloramide


In [26]:
# Elegimos las variables que queremos conservar en el dataframe final
edlist_echa =  edlist_echa[['Name_Edlist_Echa','Fuente_original','Appears on lists', 'Health effects','CAS Number', 'EC Number'  ]]

In [27]:
edlist_echa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name_Edlist_Echa  295 non-null    object 
 1   Fuente_original   295 non-null    object 
 2   Appears on lists  239 non-null    object 
 3   Health effects    239 non-null    float64
 4   CAS Number        282 non-null    object 
 5   EC Number         250 non-null    object 
dtypes: float64(1), object(5)
memory usage: 14.0+ KB


In [28]:
# Consulatamos valores únicos
edlist_echa['Name_Edlist_Echa'].nunique(dropna=True)

260

In [29]:
# Consulatamos valores únicos
edlist_echa['CAS Number'].nunique(dropna=True)

247

In [30]:
# Consulatamos valores únicos
edlist_echa['EC Number'].nunique(dropna=True)

227

Con esto sabemos que el listado máximo que podemos tener de disruptores es de 295.

Según los identificadores hay 247 y 227 pero estos pueden estar conbinados.

## 6. Segundo Merge

La unión se realizará con un left join por 'CAS Number'

In [31]:
# leemos los datos de la lista de sutanncias activas
pesticidas = pd.read_parquet("../../data/processed/notebooks/pesticidas.parquet")
pesticidas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Name_Pesticida  100 non-null    object
 1   CAS Number      100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [32]:
# Realizamos unión con la lista de pesticidas

edlist_echa_pesticidas = edlist_echa.merge(pesticidas, on=['CAS Number'], how='left' )
edlist_echa_pesticidas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name_Edlist_Echa  295 non-null    object 
 1   Fuente_original   295 non-null    object 
 2   Appears on lists  239 non-null    object 
 3   Health effects    239 non-null    float64
 4   CAS Number        282 non-null    object 
 5   EC Number         250 non-null    object 
 6   Name_Pesticida    3 non-null      object 
dtypes: float64(1), object(6)
memory usage: 16.3+ KB


In [33]:
# Devuelve True si hay al menos un None en string en esa columna
edlist_echa_pesticidas.isin(["None"]).any()

Name_Edlist_Echa    False
Fuente_original     False
Appears on lists    False
Health effects      False
CAS Number          False
EC Number           False
Name_Pesticida      False
dtype: bool

In [34]:
edlist_echa_pesticidas [(edlist_echa_pesticidas['Name_Pesticida'].notna())]

,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects,CAS Number,EC Number,Name_Pesticida
50,Cyprodinil,Edlist_2,List II,1.0,121552-61-2,601-785-8,Cyprodinil
203,Clofentezine,Edlist_1,List I,1.0,74115-24-5,277-728-2,Clofentezine
230,"p-(1,1-dimethylpropyl)phenol",Edlist_1,List I,0.0,80-46-6,201-280-9,4-t-Pentylphenol


In [35]:
disruptores_clean = edlist_echa_pesticidas[edlist_echa_pesticidas['Name_Pesticida'].isna()]
disruptores_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292 entries, 0 to 294
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name_Edlist_Echa  292 non-null    object 
 1   Fuente_original   292 non-null    object 
 2   Appears on lists  236 non-null    object 
 3   Health effects    236 non-null    float64
 4   CAS Number        279 non-null    object 
 5   EC Number         247 non-null    object 
 6   Name_Pesticida    0 non-null      object 
dtypes: float64(1), object(6)
memory usage: 18.2+ KB


### 6.1. Exportación del Segundo Merge para otras fuentes de búsquedas del nombre INCI.

In [36]:
# Elegimos las variables que queremos conservar en el dataframe final
disruptores_clean = disruptores_clean[['Name_Edlist_Echa', 'Fuente_original', 'Appears on lists', 'Health effects', 'CAS Number', 'EC Number']]

In [37]:
# Exportamos primera unión para otro Merge posterior
disruptores_clean.to_parquet("../../data/processed/notebooks/disruptores_clean.parquet", index=False)

##  7. Importamos COSING_clean

In [38]:
# leemos los datos de la lista de CosIng_clean
cosing_clean= pd.read_parquet("../../data/processed/notebooks/cosing_clean.parquet")

In [39]:
cosing_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42540 entries, 0 to 42539
Data columns (total 8 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Chemical name                              42540 non-null  object
 1   Chemical/IUPAC Name                        39874 non-null  object
 2   Identified INGREDIENTS or substances e.g.  2768 non-null   object
 3   CAS Number                                 42203 non-null  object
 4   EC Number                                  42005 non-null  object
 5   Name of Common Ingredients Glossary        2121 non-null   object
 6   Product Type, body parts                   471 non-null    object
 7   Anexo_cosIng                               42540 non-null  object
dtypes: object(8)
memory usage: 2.6+ MB


In [40]:
# Consulatamos si hay al menos un None en string en alguna columna
cosing_clean.isin([None]).any()

Chemical name                                False
Chemical/IUPAC Name                           True
Identified INGREDIENTS or substances e.g.     True
CAS Number                                    True
EC Number                                     True
Name of Common Ingredients Glossary           True
Product Type, body parts                      True
Anexo_cosIng                                 False
dtype: bool

In [41]:
#Reemplazamos None por NaN
cosing_clean = cosing_clean.replace({None: np.nan})

In [42]:
cosing_clean['Name of Common Ingredients Glossary'].nunique(dropna=True)

541

Tenemos 541 nombres de la variable que nos da el nombre INCI

## 8. Consultas antes de segunda unión de dataframes

Vamos a consultar el número de veces que se repite el valor de CAS Number y EC Number en cada tabla antes de unir, para hacernos una idea de lo que puede ocurrir con la unión.

En esta segunda unión van a haber muchos duplicados por la condición de CAS Number y EC Number en ambas tablas.

In [43]:
cosing_clean['CAS Number'].value_counts().head()

CAS Number
64742-89-8 [46]     178
64742-83-2 [131]    178
64742-73-0 [107]    178
64742-66-1 [130]    178
64742-49-0 [106]    178
Name: count, dtype: int64

In [44]:
cosing_clean['EC Number'].value_counts().head()

EC Number
232-349-1 [40]    178
232-443-2 [41]    178
232-453-7 [42]    178
265-041-0 [43]    178
265-042-6 [44]    178
Name: count, dtype: int64

In [45]:
edlist_echa['CAS Number'].value_counts().head()

CAS Number
9016-45-9     6
 541-02-6     4
 556-67-2     4
69430-24-6    4
 540-97-6     4
Name: count, dtype: int64

In [46]:
edlist_echa['EC Number'].value_counts().head()

EC Number
 208-764-9    4
 209-136-7    4
614-966-1     4
 208-762-8    4
231-212-3     3
Name: count, dtype: int64

Consultamos tambien cuantos registros que contengan ("EC Number" o "CAS Number") de la Edlist_Echa están en CosIng_clean y viceversa

In [47]:
edlist_echa[
    edlist_echa["EC Number"].isin(cosing_clean["EC Number"]) |
    edlist_echa["CAS Number"].isin(cosing_clean["CAS Number"])
].count()

Name_Edlist_Echa    108
Fuente_original     108
Appears on lists     90
Health effects       90
CAS Number           99
EC Number            64
dtype: int64

In [48]:
cosing_clean[
    cosing_clean["CAS Number" ].isin(edlist_echa["CAS Number"]) |
    cosing_clean["EC Number"   ].isin(edlist_echa["EC Number"])
    ].count()

Chemical name                                941
Chemical/IUPAC Name                          746
Identified INGREDIENTS or substances e.g.    215
CAS Number                                   604
EC Number                                    406
Name of Common Ingredients Glossary          158
Product Type, body parts                      37
Anexo_cosIng                                 941
dtype: int64

Consultamos nulos, pues esto será un problema a la hora de la unión, pues cuando hay nulos en la clave a unir, esos nulos se emparejan de forma erronea.

Aunque la eliminación de los registros que cumplen la condición de 'CAS Number' y 'CAS Number' sean nulos se hizo en los dataframes de origen, lo comprobamos.

In [49]:
edlist_echa[(edlist_echa['CAS Number'].isna() & edlist_echa['EC Number'].isna())]

,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects,CAS Number,EC Number


In [50]:
cosing_clean[(cosing_clean['CAS Number'].isna() & cosing_clean['EC Number'].isna())]

,Chemical name,Chemical/IUPAC Name,Identified INGREDIENTS or substances e.g.,CAS Number,EC Number,Name of Common Ingredients Glossary,"Product Type, body parts",Anexo_cosIng


Como la unión que queremos hacer no es como la anterior, es decir un full join , que daba igual si una de las dos claves tenía nulos, mientras la otra no la tuviera.

En esta ocasión no podemos eliminar los nulos de cada variable, porque nos quedaríamos con la posibilidad de unir por la otra clave.

Consultamos igualmente los nulos de estas variables en cada dataframe

In [51]:
edlist_echa[["EC Number", "CAS Number"]].isnull().sum()


EC Number     45
CAS Number    13
dtype: int64

In [52]:
cosing_clean[["EC Number", "CAS Number"]].isnull().sum()


EC Number     535
CAS Number    337
dtype: int64

## 9. Tercer Merge - Unión final

**La unión se realizará de la siguiente manera:**

1. Inner join por 'CAS Number', solo nos quedamos ocn los registros coincidentes en ambas tablas. El problema es que hay CAS Number nulos que no podemos eliminar, que haran emparejamientos erroneos.
2. Eliminación de 'CAS Number' nulos.
3. Inner join por 'EC Number' pasa lo mismo que con la unión anterior.
4. Eliminación de 'EC Number' nulos.
5. Concatenación de ambas tablas.

### 9.1. Unión por CAS Number

In [53]:
union_inner_cas= disruptores_clean.merge( cosing_clean, on=['CAS Number'],  how='inner', suffixes=('', '_cosing'))
union_inner_cas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4444 entries, 0 to 4443
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Name_Edlist_Echa                           4444 non-null   object 
 1   Fuente_original                            4444 non-null   object 
 2   Appears on lists                           3078 non-null   object 
 3   Health effects                             3078 non-null   float64
 4   CAS Number                                 63 non-null     object 
 5   EC Number                                  4443 non-null   object 
 6   Chemical name                              4444 non-null   object 
 7   Chemical/IUPAC Name                        4094 non-null   object 
 8   Identified INGREDIENTS or substances e.g.  231 non-null    object 
 9   EC Number_cosing                           4443 non-null   object 
 10  Name of Common Ingredien

In [54]:
# Devuelve True si hay al menos un None en string en esa columna
union_inner_cas.isin([None]).any()

Name_Edlist_Echa                             False
Fuente_original                              False
Appears on lists                             False
Health effects                               False
CAS Number                                    True
EC Number                                    False
Chemical name                                False
Chemical/IUPAC Name                          False
Identified INGREDIENTS or substances e.g.    False
EC Number_cosing                             False
Name of Common Ingredients Glossary          False
Product Type, body parts                     False
Anexo_cosIng                                 False
dtype: bool

In [55]:
union_inner_cas = union_inner_cas.replace({None: np.nan})

Como esperabamos, se han creado registros con CAS Number vacios.

Solo se ha realizado la unión correcta de 69 registros.

Vemos que cuando no CAS Number es no nulo, el emparejamiento es correcto, pues EC Number y EC Number_cosing coinciden y las variables relativas a nombres de ambas tablas tambien.

In [56]:
union_inner_cas[(union_inner_cas['CAS Number'].notna())].head()

,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects,CAS Number,EC Number,Chemical name,Chemical/IUPAC Name,Identified INGREDIENTS or substances e.g.,EC Number_cosing,Name of Common Ingredients Glossary,"Product Type, body parts",Anexo_cosIng
0,Triclocarban,Edlist_3,List III,1.0,101-20-2,202-924-1,"1-(4-Chlorophenyl)-3-(3,4-dichlorophenyl)urea",NaN,"1-(4-CHLOROPHENYL)-3-(3,4-DICHLOROPHENYL)UREA ...",202-924-1,Triclocarban,"All cosmetic products, with the exception of m...",Anexo_5
1,Triclocarban,Edlist_3,List III,1.0,101-20-2,202-924-1,"1-(4-Chlorophenyl)-3-(3,4-dichlorophenyl)urea ...",NaN,"1-(4-CHLOROPHENYL)-3-(3,4-DICHLOROPHENYL)UREA\...",202-924-1,TRICLOCARBAN,Rinse-off products,Anexo_3
2,Geraniol,ECHA - ED_pendiente,NaN,NaN,106-24-1,203-377-1,"2,6-Octadien-1-ol, 3,7-dimethyl-, (2E)- \n\n",NaN,GERANIOL,203-377-1,Geraniol,NaN,Anexo_3
3,"resorcinol; 1,3-benzenediol",Edlist_2,List II,1.0,108-46-3,203-585-2,"1,3-benzenediol","1,3-benzenediol",RESORCINOL,203-585-2,RESORCINOL,(a) Hair dye substance in oxidative hair dye p...,Anexo_3
4,Melamine,ECHA - ED_pendiente,NaN,NaN,108-78-1,203-615-4,"1,3,5-triazine-2,4,6-triamine; melamine",NaN,NaN,203-615-4,NaN,NaN,Anexo_2


Sin embargo cuando es CAS Number es NULO, el emparejamiento es errático, pues EC Number y EC Number_cosing NO coinciden y las variables relativas a nombres TAMPOCO

In [57]:
union_inner_cas[(union_inner_cas['CAS Number'].isna())].head(3)

,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects,CAS Number,EC Number,Chemical name,Chemical/IUPAC Name,Identified INGREDIENTS or substances e.g.,EC Number_cosing,Name of Common Ingredients Glossary,"Product Type, body parts",Anexo_cosIng
63,Oligomerisation and alkylation reaction produc...,ECHA - ED_Env,NaN,NaN,NaN,700-960-7,"(Methylenebis(4,1-phenylenazo(1-(3-(dimethylam...",NaN,NaN,401-500-5,NaN,NaN,Anexo_2
64,Oligomerisation and alkylation reaction produc...,ECHA - ED_Env,NaN,NaN,NaN,700-960-7,"1-(1,2,3,4,5,6,7,8-octahydro-2,3,8,8-tetrameth...",NaN,TETRAMETHYL ACETYLOCTAHYDRONAPHTHALENES,259-175-9,Tetramethyl acetyloctahydronaphthalenes,NaN,Anexo_3
65,Oligomerisation and alkylation reaction produc...,ECHA - ED_Env,NaN,NaN,NaN,700-960-7,"1-(1,2,3,4,5,6,7,8-octahydro-2,3,8,8-tetrameth...",NaN,TETRAMETHYL ACETYLOCTAHYDRONAPHTHALENES,268-978-3,Tetramethyl acetyloctahydronaphthalenes,NaN,Anexo_3


Procedemos a eliminar las variables donde CAS Number es nulo

In [58]:
union_inner_cas= union_inner_cas[~(union_inner_cas['CAS Number'].isna())]

### 9.2. Unión por EC Number

In [59]:
union_inner_ec= edlist_echa.merge( cosing_clean, on=['EC Number'],  how='inner', suffixes=('', '_cosing'))
union_inner_ec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24138 entries, 0 to 24137
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Name_Edlist_Echa                           24138 non-null  object 
 1   Fuente_original                            24138 non-null  object 
 2   Appears on lists                           23583 non-null  object 
 3   Health effects                             23583 non-null  float64
 4   CAS Number                                 24138 non-null  object 
 5   EC Number                                  63 non-null     object 
 6   Chemical name                              24138 non-null  object 
 7   Chemical/IUPAC Name                        17587 non-null  object 
 8   Identified INGREDIENTS or substances e.g.  7148 non-null   object 
 9   CAS Number_cosing                          24137 non-null  object 
 10  Name of Common Ingredi

De la misma manera, se han creado 24.138 registros pero solo son válidos 63 EC Number

In [60]:
union_inner_ec[(union_inner_ec['EC Number'].isna())].head(4)

,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects,CAS Number,EC Number,Chemical name,Chemical/IUPAC Name,Identified INGREDIENTS or substances e.g.,CAS Number_cosing,Name of Common Ingredients Glossary,"Product Type, body parts",Anexo_cosIng
0,4-(5-methylhexyl)phenol,Edlist_1,List I,0.0,100532-36-3,NaN,(3R)-beta-4-Caroten-3-ol,(3R)-beta-4-Caroten-3-ol,CI 75135,3763-55-1,CI 75135,NaN,Anexo_4
1,4-(5-methylhexyl)phenol,Edlist_1,List I,0.0,100532-36-3,NaN,"1,3,5-Triazine, 2,4,6-tris(1,1'-biphenyl)-4-yl...","1,3,5-Triazine, 2,4,6-tris(1,1'-biphenyl)-4-yl-",NaN,31274-51-8,TRIS-BIPHENYL TRIAZINE / TRIS-BIPHENYL TRIAZIN...,NaN,Anexo_6
2,4-(5-methylhexyl)phenol,Edlist_1,List I,0.0,100532-36-3,NaN,"1-(1,2,3,4,5,6,7,8-octahydro-2,3,8,8-tetrameth...",NaN,TETRAMETHYL ACETYLOCTAHYDRONAPHTHALENES,54464-59-4,Tetramethyl acetyloctahydronaphthalenes,NaN,Anexo_3
3,4-(5-methylhexyl)phenol,Edlist_1,List I,0.0,100532-36-3,NaN,"1-(1,2,3,4,5,6,7,8-octahydro-2,3,8,8-tetrameth...",NaN,TETRAMETHYL ACETYLOCTAHYDRONAPHTHALENES,68155-66-8,Tetramethyl acetyloctahydronaphthalenes,NaN,Anexo_3


In [61]:
union_inner_ec= union_inner_ec[~(union_inner_ec['EC Number'].isna())]

In [62]:
union_inner_ec.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 535 to 24137
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Name_Edlist_Echa                           63 non-null     object 
 1   Fuente_original                            63 non-null     object 
 2   Appears on lists                           43 non-null     object 
 3   Health effects                             43 non-null     float64
 4   CAS Number                                 63 non-null     object 
 5   EC Number                                  63 non-null     object 
 6   Chemical name                              63 non-null     object 
 7   Chemical/IUPAC Name                        37 non-null     object 
 8   Identified INGREDIENTS or substances e.g.  38 non-null     object 
 9   CAS Number_cosing                          62 non-null     object 
 10  Name of Common Ingredients G

### 9.3. Unión de ambas tablas

In [63]:
# Concatenamos
union_completa = pd.concat([union_inner_cas,union_inner_ec], ignore_index=True)

In [64]:
union_completa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 14 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Name_Edlist_Echa                           126 non-null    object 
 1   Fuente_original                            126 non-null    object 
 2   Appears on lists                           88 non-null     object 
 3   Health effects                             88 non-null     float64
 4   CAS Number                                 126 non-null    object 
 5   EC Number                                  125 non-null    object 
 6   Chemical name                              126 non-null    object 
 7   Chemical/IUPAC Name                        75 non-null     object 
 8   Identified INGREDIENTS or substances e.g.  74 non-null     object 
 9   EC Number_cosing                           62 non-null     object 
 10  Name of Common Ingredients

In [65]:
union_completa['Name_Edlist_Echa'].nunique(dropna=True)

56

In [66]:
union_completa['Name of Common Ingredients Glossary'].nunique(dropna=True)

17

De los 126 registros hay 56 registros únicos, de los cuales 17 registros son de la variable que nos interesa 'Name of Common Ingredients Glossary'.

El resto son duplicados

## 10. Comprobaciones del 3º Merge

Para verificar que la unión se ha realizado correctamente, vamos a comprobar que los CAS Number y EC Number originales coindidan con los creados en la unión CAS Number_cosing y EC Number_cosing

In [67]:
Consulta_1 = union_completa[(union_completa['EC Number'].notna() & union_completa['EC Number_cosing'].notna())]
Consulta_1[['EC Number', 'EC Number_cosing']].head()

,EC Number,EC Number_cosing
0,202-924-1,202-924-1
1,202-924-1,202-924-1
2,203-377-1,203-377-1
3,203-585-2,203-585-2
4,203-615-4,203-615-4


In [68]:
Consulta_2 = union_completa[(union_completa['CAS Number'].notna() & union_completa['CAS Number_cosing'].notna())]
Consulta_2[['CAS Number', 'CAS Number_cosing',]].head()

,CAS Number,CAS Number_cosing
63,101-20-2,101-20-2
64,101-20-2,101-20-2
65,106-24-1,106-24-1
66,108-46-3,108-46-3
67,108-78-1,108-78-1


A simple vista parece que coinciden las variables en origen con las variables creadas.

Ahora vamos a comprobar si hay valores distintos y cuál es su comportamiento.

Para ello, vamos a filtrar por CAS/EC Number distinto pero vamos a ver las variables nombre, tanto las que proceden de la tabla anterior edlist_echa como de cosing. De esta forma podemos ver si tienen el mismo nombre aunque tengan distintos CAS/EC Number

In [69]:
# Consulta para CAS Number distintos
diferentes_cas = union_completa[
    (union_completa['CAS Number'].notna()) &
    (union_completa['CAS Number_cosing'].notna()) &
    (union_completa['CAS Number'] != union_completa['CAS Number_cosing'])
]

diferentes_cas[['CAS Number', 'CAS Number_cosing','EC Number', 'EC Number_cosing', 'Name_Edlist_Echa', 'Chemical name', 
'Chemical/IUPAC Name', 'Name of Common Ingredients Glossary', 'Identified INGREDIENTS or substances e.g.' ]].head(3)

,CAS Number,CAS Number_cosing,EC Number,EC Number_cosing,Name_Edlist_Echa,Chemical name,Chemical/IUPAC Name,Name of Common Ingredients Glossary,Identified INGREDIENTS or substances e.g.
91,36861-47-9,38102-62-4,253-242-6,NaN,"(±)-1,7,7-trimethyl-3-[(4-methylphenyl)methyle...",3-(4'-methylbenzylidene)-camphor; [INCI: 4-Met...,NaN,NaN,4-METHYLBENZYLIDENE CAMPHOR
104,7440-50-8,7440-50-8,231-159-6,NaN,Copper,Copper,Copper,CI 77400,CI 77400
118,94-26-8,35285-69-9,202-318-7,NaN,Butyl 4-hydroxybenzoate; Butylparaben,Butyl 4-hydroxy­benzoate and its salts Propyl ...,Butyl 4-hydroxy­benzoate and its salts Propyl ...,BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARAB...,BUTYLPARABEN\nPOTASSIUM BUTYLPARABEN\nPOTASSIU...


Parece que el primer registro de la consulta no se ha realizado bien la unión. 

Las 4 variables que hacen referencia al nombre no coinciden 

Consultamos los nombres de las 2 tablas.

A simple vista parece que no, pero consultando el EC Number por el que se ha unido parece que se trata de la misma sustancia si consultamos la fuente de origen cosing_clean.

Se trata del mismo producto, que tiene mismo EC Number y distinto CAS Number, esto es debido al método explode utilizado para poder tener un solo número por fila.

In [70]:
cosing_clean.loc[cosing_clean["EC Number"] == "253-242-6", 
                 ['CAS Number','Chemical name']]


,CAS Number,Chemical name
675,36861-47-9,3-(4'-methylbenzylidene)-camphor; [INCI: 4-Met...
677,38102-62-4,3-(4'-methylbenzylidene)-camphor; [INCI: 4-Met...


In [71]:
# Consulta para EC Number distintos
diferentes_ec = union_completa[
    (union_completa['EC Number'].notna()) &
    (union_completa['EC Number_cosing'].notna()) &
    (union_completa['EC Number'] != union_completa['EC Number_cosing'])
]

diferentes_ec[['CAS Number', 'CAS Number_cosing','EC Number', 'EC Number_cosing', 'Name_Edlist_Echa', 'Chemical name','Chemical/IUPAC Name', 'Name of Common Ingredients Glossary', 'Identified INGREDIENTS or substances e.g.' ]]

,CAS Number,CAS Number_cosing,EC Number,EC Number_cosing,Name_Edlist_Echa,Chemical name,Chemical/IUPAC Name,Name of Common Ingredients Glossary,Identified INGREDIENTS or substances e.g.
5,110235-47-7,NaN,432-140-7,600-951-7,Mepanipyrim,Mepanipyrim; 4-methyl-N-phenyl-6-(1-propynyl)-...,Mepanipyrim; 4-methyl-N-phenyl-6-(1-propynyl)-...,NaN,NaN
55,94-26-8,NaN,202-318-7,202-307-7,Butyl 4-hydroxybenzoate; Butylparaben,Butyl 4-hydroxy­benzoate and its salts Propyl ...,Butyl 4-hydroxy­benzoate and its salts Propyl ...,BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARAB...,BUTYLPARABEN\nPOTASSIUM BUTYLPARABEN\nPOTASSIU...
56,94-26-8,NaN,202-318-7,252-488-1,Butyl 4-hydroxybenzoate; Butylparaben,Butyl 4-hydroxy­benzoate and its salts Propyl ...,Butyl 4-hydroxy­benzoate and its salts Propyl ...,BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARAB...,BUTYLPARABEN\nPOTASSIUM BUTYLPARABEN\nPOTASSIU...
57,94-26-8,NaN,202-318-7,253-049-7,Butyl 4-hydroxybenzoate; Butylparaben,Butyl 4-hydroxy­benzoate and its salts Propyl ...,Butyl 4-hydroxy­benzoate and its salts Propyl ...,BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARAB...,BUTYLPARABEN\nPOTASSIUM BUTYLPARABEN\nPOTASSIU...
58,94-26-8,NaN,202-318-7,254-009-1,Butyl 4-hydroxybenzoate; Butylparaben,Butyl 4-hydroxy­benzoate and its salts Propyl ...,Butyl 4-hydroxy­benzoate and its salts Propyl ...,BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARAB...,BUTYLPARABEN\nPOTASSIUM BUTYLPARABEN\nPOTASSIU...
59,94-26-8,NaN,202-318-7,284-597-5,Butyl 4-hydroxybenzoate; Butylparaben,Butyl 4-hydroxy­benzoate and its salts Propyl ...,Butyl 4-hydroxy­benzoate and its salts Propyl ...,BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARAB...,BUTYLPARABEN\nPOTASSIUM BUTYLPARABEN\nPOTASSIU...


En este caso, parece todo en orden.

In [72]:
nombre_mas_informativo_generico(union_completa, 'Chemical name', 'Chemical/IUPAC Name', 'Name_Chemical_Cosing')
union_completa[['Chemical name', 'Chemical/IUPAC Name', 'Name_Chemical_Cosing']].iloc[17:26]

,Chemical name,Chemical/IUPAC Name,Name_Chemical_Cosing
17,Tris[2-chloro-1-(chloromethyl)ethyl] phosphate,Tris[2-chloro-1-(chloromethyl)ethyl] phosphate,Tris[2-chloro-1-(chloromethyl)ethyl] phosphate
18,Ziram,NaN,Ziram
19,3-methylpyrazole,3-methylpyrazole,3-methylpyrazole
20,"4,4'-[2,2,2-trifluoro-1-(trifluoromethyl)ethyl...",NaN,"4,4'-[2,2,2-trifluoro-1-(trifluoromethyl)ethyl..."
21,3-Benzylidene camphor,NaN,3-Benzylidene camphor
22,Propargite (ISO),2-(-tert-butylphenoxy)cyclohexyl prop-2-ynyl s...,2-(-tert-butylphenoxy)cyclohexyl prop-2-ynyl s...
23,Thiophanate-methyl,"1,2-di-(3-methoxycarbonyl-2-thioureido)benzene","1,2-di-(3-methoxycarbonyl-2-thioureido)benzene"
24,"1,2,4-Triazole",NaN,"1,2,4-Triazole"
25,Diuron (ISO),"3-(3,4-dichlorophenyl)-1,1-dimethylurea","3-(3,4-dichlorophenyl)-1,1-dimethylurea"


He consultado manualmente en la web de COSING y cuando no hay nombre "Name of Common Ingredients Glossary" su nombre INCI es el de la variable "Identified INGREDIENTS or substances e.g."

In [73]:
# Combinamos las vatiables de nombre 
union_completa["nombre_etiqueta"] = union_completa["Name of Common Ingredients Glossary"].combine_first(
    union_completa["Identified INGREDIENTS or substances e.g."]
)

In [74]:
# Elegimos las variables que hacen referencia a nombres y las que nos van a servir más adelante para otros fines.

union_completa =  union_completa[['Name_Edlist_Echa', 'Fuente_original', 'Appears on lists', 'Health effects','CAS Number', 'EC Number', 
'Name_Chemical_Cosing', 'nombre_etiqueta', 'Product Type, body parts','Anexo_cosIng' ]]

In [75]:
union_completa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name_Edlist_Echa          126 non-null    object 
 1   Fuente_original           126 non-null    object 
 2   Appears on lists          88 non-null     object 
 3   Health effects            88 non-null     float64
 4   CAS Number                126 non-null    object 
 5   EC Number                 125 non-null    object 
 6   Name_Chemical_Cosing      126 non-null    object 
 7   nombre_etiqueta           74 non-null     object 
 8   Product Type, body parts  20 non-null     object 
 9   Anexo_cosIng              126 non-null    object 
dtypes: float64(1), object(9)
memory usage: 10.0+ KB


In [76]:
# Devuelve True si hay al menos un None en string en esa columna
union_completa.isin(["None"]).any()

Name_Edlist_Echa            False
Fuente_original             False
Appears on lists            False
Health effects              False
CAS Number                  False
EC Number                   False
Name_Chemical_Cosing        False
nombre_etiqueta             False
Product Type, body parts    False
Anexo_cosIng                False
dtype: bool

## 11. Variable "nombre_etiqueta": tratamiento de símbolos y duplicados

Ahora vamos a comprobar la varible 'nombre_etiqueta', porque es la variable objetivo, pues es de la que obtendremos el nombre de la etiqueta.

Vamos a ver si esta variable tiene más de un nombre en un registro separado por comas, barras o punto y coma.

Los guiones y las comas no podemos tenerlos en cuenta porque claramente forman parte del nombre, y aunque parece más un nombre químico se encuentra en esta variable.

In [77]:
union_completa['nombre_etiqueta'].value_counts()

nombre_etiqueta
BUTYLPARABEN/PROPYLPARABEN/SODIUM PROPOYLPARABEN/SODIUM BUTYLPARABEN/POTASSIUM BUTYLPARABEN/POTASSIUM PROPYLPARABEN    12
4-METHYLBENZYLIDENE CAMPHOR                                                                                             4
Geraniol                                                                                                                2
Triclocarban                                                                                                            2
DIETHYLHEXYL PHTHALATE                                                                                                  2
METHYL SALICYLATE                                                                                                       2
Hexamethylindanopyran                                                                                                   2
TRICLOCARBAN                                                                                                            2
Butylate

A simple vista vemos que hay ingredientes en **nombre_etiqueta* que contienen más de un ingrediente pero están separados por el símbolo /

Vamos a dividir y explotar la variable

In [78]:
# Separamos nombre_etiqueta por /
union_completa['nombre_etiqueta'] = union_completa['nombre_etiqueta'].str.split(r'[/]')

In [79]:
# Descomponemos fila por nombre_etiqueta
union_completa = union_completa.explode('nombre_etiqueta')

In [80]:
union_completa['nombre_etiqueta'].value_counts()

nombre_etiqueta
SODIUM PROPOYLPARABEN                        12
PROPYLPARABEN                                12
BUTYLPARABEN                                 12
POTASSIUM PROPYLPARABEN                      12
POTASSIUM BUTYLPARABEN                       12
SODIUM BUTYLPARABEN                          12
4-METHYLBENZYLIDENE CAMPHOR                   4
DIETHYLHEXYL PHTHALATE                        2
TRICLOCARBAN                                  2
Triclocarban                                  2
RESORCINOL                                    2
Geraniol                                      2
METHYL SALICYLATE                             2
Triclosan                                     2
2-BROMO-2-NITROPROPANE-1,3-DIOL               2
Butylated Hydroxytoluene                      2
Hexamethylindanopyran                         2
BENZOPHENONE-3                                2
TRICRESYL PHOSPHATE                           2
3-BENZYLIDENE CAMPHOR                         2
AMMONIUM SILVER ZINC ALU

In [81]:
union_completa.isnull().sum()

Name_Edlist_Echa              0
Fuente_original               0
Appears on lists             38
Health effects               38
CAS Number                    0
EC Number                     1
Name_Chemical_Cosing          0
nombre_etiqueta              52
Product Type, body parts    166
Anexo_cosIng                  0
dtype: int64

In [82]:
# pasamos a minusculas la variable nombre_etiqueta
union_completa['nombre_etiqueta'] = union_completa['nombre_etiqueta'].str.lower()


In [83]:
union_completa = union_completa.drop_duplicates(subset='nombre_etiqueta', keep='first')

## 12. Lista de disruptores con nombre de etiqueta de este proceso de ETL

In [84]:
union_completa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 0 to 104
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name_Edlist_Echa          37 non-null     object 
 1   Fuente_original           37 non-null     object 
 2   Appears on lists          26 non-null     object 
 3   Health effects            26 non-null     float64
 4   CAS Number                37 non-null     object 
 5   EC Number                 37 non-null     object 
 6   Name_Chemical_Cosing      37 non-null     object 
 7   nombre_etiqueta           36 non-null     object 
 8   Product Type, body parts  9 non-null      object 
 9   Anexo_cosIng              37 non-null     object 
dtypes: float64(1), object(9)
memory usage: 3.2+ KB


**LISTA DEFINITIVA DE ESTE ETL**

In [85]:
disruptores_etiqueta = union_completa[(union_completa['nombre_etiqueta'].notnull())]

# Dejamos las variables que necesitamos
disruptores_etiqueta = disruptores_etiqueta[['Name_Edlist_Echa', 'Name_Chemical_Cosing','Fuente_original','Anexo_cosIng','Appears on lists', 'Health effects','CAS Number','EC Number','nombre_etiqueta']]
disruptores_etiqueta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 104
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      36 non-null     object 
 1   Name_Chemical_Cosing  36 non-null     object 
 2   Fuente_original       36 non-null     object 
 3   Anexo_cosIng          36 non-null     object 
 4   Appears on lists      26 non-null     object 
 5   Health effects        26 non-null     float64
 6   CAS Number            36 non-null     object 
 7   EC Number             36 non-null     object 
 8   nombre_etiqueta       36 non-null     object 
dtypes: float64(1), object(8)
memory usage: 2.8+ KB


In [86]:
# Exportamos 
disruptores_etiqueta.to_excel("../../data/processed/notebooks/disruptores_etiqueta.xlsx", index=False)
disruptores_etiqueta.to_parquet("../../data/processed/notebooks/disruptores_etiqueta.parquet", index=False)